![Nested chat workflow](image/nested_chats.png)

## Setup

In [63]:
import autogen

config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["llama-3.2-1b-instruct"]
    }
)

llm_config = {
    "cache_seed": 43,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,  # in seconds
}

## Create a writer agent

In [64]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''

In [65]:
writer = autogen.AssistantAgent(
    name="Writer",
    system_message=(
        "You are a professional blog writer.\n"
        "Your job is to write an engaging and concise blog post with a title, based on a given topic.\n\n"
        "You will receive feedback after your first version. Based on that feedback, revise and improve your blog post.\n"
        "Your final output must be a polished blog post (title + content), with no explanations, commentary, or repetition of the feedback.\n\n"
        "**Important:** Only return the rewritten blog post — not the feedback summary or any notes."
    ),
    llm_config=llm_config,
)

In [66]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])
print(reply)

**Unlocking AI Potential: The Power of Deep Learning**

Deep learning has revolutionized the field of artificial intelligence, enabling machines to learn from vast amounts of data and make predictions with unprecedented accuracy. This technology has far-reaching applications in image recognition, natural language processing, and predictive modeling. By leveraging neural networks, deep learning algorithms can analyze complex patterns and relationships, leading to breakthroughs in fields like healthcare, finance, and transportation. As AI continues to evolve, the possibilities for innovation are endless, and deep learning is at the forefront of this exciting journey.


## Nested chat

In [67]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
    "your ability to optimize content for search engines, "
    "ensuring that it ranks well and attracts organic traffic. "
    "Make sure your suggestion is concise (within 3 bullet points), "
    "concrete and to the point. "
    "Begin the review by stating your role.",
)
legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
    "your ability to ensure that content is legally compliant "
    "and free from any potential legal issues. "
    "Make sure your suggestion is concise (within 3 bullet points), "
    "concrete and to the point. "
    "Begin the review by stating your role.",
)
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
    "your ability to ensure that content is ethically sound "
    "and free from any potential ethical issues. "
    "Make sure your suggestion is concise (within 3 bullet points), "
    "concrete and to the point. "
    "Begin the review by stating your role. ",
)
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

In [68]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''


review_chats = [
    {
        "recipient": SEO_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt":
                         "Return review into as JSON object only:"
                         "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role", },
        "max_turns": 1},
    {
        "recipient": legal_reviewer, "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt":
                         "Return review into as JSON object only:"
                         "{'Reviewer': '', 'Review': ''}.", },
        "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt":
                      "Return review into as JSON object only:"
                      "{'reviewer': '', 'review': ''}", },
     "max_turns": 1},
    {"recipient": meta_reviewer,
     "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
     "summary_method": "last_msg",
     "max_turns": 1},
]

In [69]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
    "the writer and provide constructive "
    "feedback to help improve the quality of the content.",
)

critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

res = critic.initiate_chat(
    recipient=writer, message=task, summary_method="last_msg", max_turns=3)

Critic (to Writer):




        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking AI Potential: The Power of Deep Learning**

Deep learning has revolutionized the field of artificial intelligence, enabling machines to learn from vast amounts of data and make predictions with unprecedented accuracy. This technology has far-reaching applications in image recognition, natural language processing, and predictive modeling. By leveraging neural networks, deep learning algorithms can analyze complex patterns and relationships, leading to breakthroughs in fields like healthcare, finance, and transportation. As AI continues to evolve, the possibilities for innovation are endless, and deep learning is at the forefront of this exciting journey.

--------------------------------------------------------------------------------

****

In [70]:
print(res.summary)

Here is the rewritten blog post with the suggested changes:

**Unlocking AI Potential: The Power of Deep Learning**

Deep learning has revolutionized the field of artificial intelligence, enabling machines to learn from vast amounts of data and make predictions with unprecedented accuracy. This technology has far-reaching applications in image recognition, natural language processing, and predictive modeling.

By leveraging neural networks, deep learning algorithms can analyze complex patterns and relationships, leading to breakthroughs in fields like healthcare, finance, and transportation. For instance, deep learning-powered computer vision systems have enabled self-driving cars to navigate complex road networks with ease, while natural language processing tools are being used to improve customer service chatbots.

As AI continues to evolve, the possibilities for innovation are endless, and deep learning is at the forefront of this exciting journey.

**Meta Description:** "Discover h